# Lambda calculus in Kotlin

## Basic interface definition

Since `kotlin.FunctionN` is not able to express a recursive lambda type, whose parameter and return type are both itself, we define a new interface `Lambda` to express it.

In [12]:
fun interface Lambda {
    operator fun invoke(f: Lambda): Lambda
}

## Shorthand for defining a lambda expression with currying

If we want to write `λab.a` in Kotlin, embedded braces like `{ a -> { b -> a } }` are annoying.
Thus, we can use the power of currying.

In [13]:
inline fun lambda(crossinline block: (Lambda) -> Lambda) =
    Lambda { p0 -> block(p0) }

inline fun lambda(crossinline block: (Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1 -> block(p0, p1) } }

inline fun lambda(crossinline block: (Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2 -> block(p0, p1, p2) } }

inline fun lambda(crossinline block: (Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3 -> block(p0, p1, p2, p3) } }

inline fun lambda(crossinline block: (Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4 -> block(p0, p1, p2, p3, p4) } }

inline fun lambda(crossinline block: (Lambda, Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4, p5 -> block(p0, p1, p2, p3, p4, p5) } }

## Natural numbers

Then we can define natural numbers naturally.

In [14]:
val zero = lambda { f, x -> x }
val one = lambda { f, x -> f(x) }
val two = lambda { f, x -> f(f(x)) }

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[14], line 1, column 21: Parameter 'f' is never used, could be renamed to _
Exception while generating code for:
CONSTRUCTOR visibility:public returnType:<root>.Line_15_jupyter [primary]
  VALUE_PARAMETER SCRIPT_CALL_PARAMETER kind:Regular name:<earlierScripts> index:0 type:kotlin.Array<kotlin.Any>
  VALUE_PARAMETER SCRIPT_IMPLICIT_RECEIVER kind:Regular name:<this> index:1 type:jupyter.kotlin.ScriptTemplateWithDisplayHelpers
  BLOCK_BODY
    DELEGATING_CONSTRUCTOR_CALL 'public constructor <init> () [primary] declared in kotlin.Any'
    SET_FIELD 'FIELD SCRIPT_EARLIER_SCRIPTS name:$$earlierScripts type:kotlin.Array<kotlin.Any> visibility:private [final] declared in <root>.Line_15_jupyter' type=kotlin.Unit origin=null
      receiver: GET_VAR '<this>: <root>.Line_15_jupyter declared in <root>.Line_15_jupyter' type=<root>.Line_15_jupyter origin=null
      value: GET_VAR '<earlierScripts>: kotlin.Array<kotlin.Any> dec